# Mô hình Ước lượng Nỗ lực Phát triển Phần mềm Dựa trên COCOMO II

Notebook này thực hiện huấn luyện các mô hình học máy để ước lượng nỗ lực phát triển phần mềm (effort) dựa trên các tham số đầu vào của mô hình COCOMO II.

## Mục tiêu
- Kết hợp dữ liệu từ 3 schema khác nhau (LOC, FP, UCP) đã được tiền xử lý
- Huấn luyện các mô hình học máy để dự đoán effort:
  1. Linear Regression (baseline)
  2. Decision Tree Regressor
  3. Random Forest Regressor
- Đánh giá hiệu suất của các mô hình
- Xuất mô hình dưới dạng file .pkl để sử dụng trong tương lai

## Các bước thực hiện
1. Import thư viện và đọc dữ liệu
2. Kết hợp dữ liệu từ các schema khác nhau
3. Tiền xử lý dữ liệu cho quá trình huấn luyện
4. Huấn luyện và đánh giá các mô hình
5. Trực quan hóa kết quả
6. Lưu trữ mô hình

In [ ]:
# Import các thư viện cần thiết
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import joblib
import json
import warnings
warnings.filterwarnings('ignore')

# Cấu hình hiển thị
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
plt.rcParams['figure.figsize'] = (12, 8)
plt.style.use('ggplot')

# Thiết lập các đường dẫn
INPUT_DIR = '/home/huy/Huy-workspace/AI-Project/processed_data'
OUTPUT_DIR = '/home/huy/Huy-workspace/AI-Project/models'

# Tạo thư mục đầu ra cho mô hình nếu chưa tồn tại
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    print(f"Đã tạo thư mục {OUTPUT_DIR} để lưu trữ mô hình")

## 1. Đọc và Khám phá dữ liệu

Đầu tiên, chúng ta sẽ đọc dữ liệu từ các file CSV đã được tiền xử lý theo 3 schema (LOC, FP, UCP) và thực hiện khám phá dữ liệu.

In [ ]:
# Hàm đọc dữ liệu từ các file CSV
def load_data():
    """
    Đọc dữ liệu từ các file CSV đã tiền xử lý theo 3 schema
    
    Returns:
        Dictionary chứa các DataFrame
    """
    data = {}
    
    # Đọc metadata để nắm rõ cấu trúc của dữ liệu
    metadata_path = os.path.join(INPUT_DIR, 'metadata.json')
    if os.path.exists(metadata_path):
        with open(metadata_path, 'r') as f:
            metadata = json.load(f)
        print(f"Đã tải metadata từ {metadata_path}")
    else:
        metadata = {}
        print("Không tìm thấy file metadata.json")
    
    # Đọc dữ liệu LOC
    loc_path = os.path.join(INPUT_DIR, 'loc_based.csv')
    if os.path.exists(loc_path):
        data['loc'] = pd.read_csv(loc_path)
        print(f"Đã tải dữ liệu LOC: {data['loc'].shape[0]} dòng × {data['loc'].shape[1]} cột")
    else:
        print(f"Không tìm thấy file {loc_path}")
        data['loc'] = pd.DataFrame()
    
    # Đọc dữ liệu FP
    fp_path = os.path.join(INPUT_DIR, 'fp_based.csv')
    if os.path.exists(fp_path):
        data['fp'] = pd.read_csv(fp_path)
        print(f"Đã tải dữ liệu FP: {data['fp'].shape[0]} dòng × {data['fp'].shape[1]} cột")
    else:
        print(f"Không tìm thấy file {fp_path}")
        data['fp'] = pd.DataFrame()
    
    # Đọc dữ liệu UCP
    ucp_path = os.path.join(INPUT_DIR, 'ucp_based.csv')
    if os.path.exists(ucp_path):
        data['ucp'] = pd.read_csv(ucp_path)
        print(f"Đã tải dữ liệu UCP: {data['ucp'].shape[0]} dòng × {data['ucp'].shape[1]} cột")
    else:
        print(f"Không tìm thấy file {ucp_path}")
        data['ucp'] = pd.DataFrame()
    
    return data, metadata

# Tải dữ liệu
data, metadata = load_data()

# Hiển thị thông tin về mỗi schema
for schema, df in data.items():
    if not df.empty:
        print(f"\n--- Schema {schema.upper()} ---")
        print(f"Kích thước: {df.shape}")
        print("Các cột:")
        for col in df.columns:
            print(f"  - {col} ({df[col].dtype})")
        
        print("\nThống kê mô tả:")
        print(df.describe().round(2))
        
        print("\nMẫu dữ liệu:")
        print(df.head())

## 2. Kết hợp dữ liệu từ các schema

Chúng ta sẽ kết hợp dữ liệu từ các schema khác nhau thành một tập dữ liệu duy nhất để huấn luyện mô hình. Trước khi kết hợp, chúng ta cần thêm một cột "schema" để xác định nguồn của mỗi dòng dữ liệu.

In [ ]:
# Hàm chuẩn bị và kết hợp dữ liệu
def prepare_combined_data(data):
    """
    Chuẩn bị và kết hợp dữ liệu từ các schema khác nhau
    
    Args:
        data: Dictionary chứa các DataFrame của từng schema
        
    Returns:
        DataFrame đã kết hợp
    """
    dfs = []
    
    # Chuẩn bị dữ liệu LOC
    if 'loc' in data and not data['loc'].empty:
        loc_df = data['loc'].copy()
        loc_df['schema'] = 'LOC'
        loc_df['size'] = loc_df['kloc']  # Đổi tên để thống nhất
        # Thêm các cột giả để cấu trúc thống nhất với các schema khác
        if 'fp' not in loc_df.columns:
            loc_df['fp'] = np.nan
        if 'ucp' not in loc_df.columns:
            loc_df['ucp'] = np.nan
        dfs.append(loc_df)
    
    # Chuẩn bị dữ liệu FP
    if 'fp' in data and not data['fp'].empty:
        fp_df = data['fp'].copy()
        fp_df['schema'] = 'FP'
        fp_df['size'] = fp_df['fp']  # Đổi tên để thống nhất
        # Thêm các cột giả để cấu trúc thống nhất với các schema khác
        if 'kloc' not in fp_df.columns:
            fp_df['kloc'] = np.nan
        if 'ucp' not in fp_df.columns:
            fp_df['ucp'] = np.nan
        dfs.append(fp_df)
    
    # Chuẩn bị dữ liệu UCP
    if 'ucp' in data and not data['ucp'].empty:
        ucp_df = data['ucp'].copy()
        ucp_df['schema'] = 'UCP'
        ucp_df['size'] = ucp_df['ucp']  # Đổi tên để thống nhất
        # Thêm các cột giả để cấu trúc thống nhất với các schema khác
        if 'kloc' not in ucp_df.columns:
            ucp_df['kloc'] = np.nan
        if 'fp' not in ucp_df.columns:
            ucp_df['fp'] = np.nan
        dfs.append(ucp_df)
    
    # Kết hợp các DataFrame
    if dfs:
        combined_df = pd.concat(dfs, ignore_index=True)
        
        # Đảm bảo các cột cần thiết tồn tại
        required_columns = ['source', 'schema', 'size', 'effort_pm', 'time_months', 'developers']
        for col in required_columns:
            if col not in combined_df.columns:
                print(f"Cột {col} không tồn tại trong dữ liệu kết hợp!")
        
        return combined_df
    else:
        print("Không có dữ liệu nào để kết hợp!")
        return pd.DataFrame()

# Kết hợp dữ liệu
combined_df = prepare_combined_data(data)

# Hiển thị thông tin về dữ liệu kết hợp
if not combined_df.empty:
    print(f"Dữ liệu kết hợp: {combined_df.shape[0]} dòng × {combined_df.shape[1]} cột")
    print("\nPhân bố theo schema:")
    print(combined_df['schema'].value_counts())
    
    print("\nThống kê mô tả:")
    print(combined_df[['size', 'effort_pm', 'time_months', 'developers']].describe().round(2))
    
    print("\nMẫu dữ liệu:")
    print(combined_df.head())
else:
    print("Không thể kết hợp dữ liệu!")

## 3. Tiền xử lý dữ liệu cho huấn luyện mô hình

Trước khi huấn luyện mô hình, chúng ta cần thực hiện một số bước tiền xử lý:
1. Xử lý các giá trị thiếu
2. Mã hóa các biến phân loại
3. Chuẩn hóa các biến số
4. Biến đổi logarithmic (nếu cần)

In [ ]:
# Phân tích tương quan giữa các biến
if not combined_df.empty:
    plt.figure(figsize=(12, 10))
    numeric_cols = ['size', 'effort_pm', 'time_months', 'developers']
    correlation = combined_df[numeric_cols].corr()
    mask = np.triu(np.ones_like(correlation, dtype=bool))
    sns.heatmap(correlation, annot=True, fmt='.2f', cmap='coolwarm', mask=mask, cbar_kws={'shrink': .8})
    plt.title('Correlation Matrix for Combined Data')
    plt.show()
    
    # Biểu đồ phân tán theo schema
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 3, 1)
    for schema in combined_df['schema'].unique():
        subset = combined_df[combined_df['schema'] == schema]
        plt.scatter(subset['size'], subset['effort_pm'], label=schema, alpha=0.7)
    plt.xlabel('Size')
    plt.ylabel('Effort (person-months)')
    plt.title('Size vs Effort by Schema')
    plt.legend()
    
    plt.subplot(1, 3, 2)
    for schema in combined_df['schema'].unique():
        subset = combined_df[combined_df['schema'] == schema]
        plt.scatter(subset['size'], subset['time_months'], label=schema, alpha=0.7)
    plt.xlabel('Size')
    plt.ylabel('Time (months)')
    plt.title('Size vs Time by Schema')
    plt.legend()
    
    plt.subplot(1, 3, 3)
    for schema in combined_df['schema'].unique():
        subset = combined_df[combined_df['schema'] == schema]
        plt.scatter(subset['size'], subset['developers'], label=schema, alpha=0.7)
    plt.xlabel('Size')
    plt.ylabel('Developers')
    plt.title('Size vs Developers by Schema')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    # Kiểm tra phân phối của biến mục tiêu (effort_pm)
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 2, 1)
    sns.histplot(combined_df['effort_pm'], kde=True, bins=30)
    plt.title('Distribution of Effort (person-months)')
    
    plt.subplot(1, 2, 2)
    sns.histplot(np.log1p(combined_df['effort_pm']), kde=True, bins=30)
    plt.title('Distribution of log(Effort) (person-months)')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Tiền xử lý dữ liệu cho huấn luyện mô hình
def preprocess_data(df, target='effort_pm', log_transform=True):
    """
    Tiền xử lý dữ liệu cho huấn luyện mô hình
    
    Args:
        df: DataFrame đầu vào
        target: Tên biến mục tiêu
        log_transform: Có áp dụng biến đổi logarithmic hay không
        
    Returns:
        X, y, preprocessor: Dữ liệu đã xử lý và bộ tiền xử lý
    """
    if df.empty:
        print("DataFrame đầu vào rỗng!")
        return None, None, None
    
    # Tạo bản sao để không ảnh hưởng đến dữ liệu gốc
    df_copy = df.copy()
    
    # Xác định các biến phân loại và số
    categorical_cols = ['schema']
    numeric_cols = ['size', 'time_months', 'developers']
    
    # Thêm các cột phân loại khác nếu có
    for col in ['sector', 'language', 'methodology', 'applicationtype']:
        if col in df.columns:
            categorical_cols.append(col)
    
    # Nếu có cả 3 kích thước, thêm vào làm đặc trưng
    if 'kloc' in df.columns and 'kloc' not in numeric_cols:
        numeric_cols.append('kloc')
    if 'fp' in df.columns and 'fp' not in numeric_cols:
        numeric_cols.append('fp')
    if 'ucp' in df.columns and 'ucp' not in numeric_cols:
        numeric_cols.append('ucp')
    
    # Sắp xếp lại các đặc trưng để chỉ giữ lại các cột quan trọng
    features = categorical_cols + numeric_cols
    
    # Lọc các cột tồn tại trong dữ liệu
    features = [col for col in features if col in df.columns]
    
    # Chuẩn bị dữ liệu
    X = df_copy[features]
    y = df_copy[target]
    
    # Áp dụng biến đổi logarithmic nếu cần
    if log_transform:
        y = np.log1p(y)
    
    # Thiết lập bộ tiền xử lý
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])
    
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, [col for col in numeric_cols if col in X.columns]),
            ('cat', categorical_transformer, [col for col in categorical_cols if col in X.columns])
        ],
        remainder='drop'
    )
    
    return X, y, preprocessor

# Tiền xử lý dữ liệu
X, y, preprocessor = preprocess_data(combined_df, target='effort_pm', log_transform=True)

# Phân chia tập huấn luyện và tập kiểm thử
if X is not None and y is not None:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print(f"Tập huấn luyện: {X_train.shape[0]} mẫu")
    print(f"Tập kiểm thử: {X_test.shape[0]} mẫu")
    
    # Hiển thị các đặc trưng được sử dụng
    print("\nCác đặc trưng được sử dụng:")
    for col in X.columns:
        print(f"  - {col}")

## 4. Huấn luyện và đánh giá các mô hình

Chúng ta sẽ huấn luyện và đánh giá 3 mô hình:
1. Linear Regression (baseline)
2. Decision Tree Regressor
3. Random Forest Regressor

In [ ]:
# Hàm đánh giá mô hình
def evaluate_model(model, X_test, y_test, model_name, log_transform=True):
    """
    Đánh giá hiệu suất của mô hình
    
    Args:
        model: Mô hình đã huấn luyện
        X_test: Dữ liệu kiểm thử
        y_test: Nhãn kiểm thử
        model_name: Tên mô hình
        log_transform: Biến đổi logarithmic đã được áp dụng hay chưa
        
    Returns:
        Dictionary chứa các chỉ số hiệu suất
    """
    # Dự đoán trên tập kiểm thử
    y_pred = model.predict(X_test)
    
    # Chuyển đổi ngược lại nếu đã áp dụng biến đổi logarithmic
    if log_transform:
        y_test_orig = np.expm1(y_test)
        y_pred_orig = np.expm1(y_pred)
    else:
        y_test_orig = y_test
        y_pred_orig = y_pred
    
    # Tính các chỉ số hiệu suất
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Tính các chỉ số hiệu suất trên thang đo gốc
    mse_orig = mean_squared_error(y_test_orig, y_pred_orig)
    rmse_orig = np.sqrt(mse_orig)
    mae_orig = mean_absolute_error(y_test_orig, y_pred_orig)
    r2_orig = r2_score(y_test_orig, y_pred_orig)
    
    # Tính MMRE (Mean Magnitude of Relative Error) và Pred(0.25)
    are = np.abs(y_pred_orig - y_test_orig) / y_test_orig
    mre = np.mean(are)
    pred_25 = np.mean(are <= 0.25)  # Tỷ lệ dự đoán trong khoảng 25%
    
    # In kết quả
    print(f"\n--- Kết quả đánh giá mô hình {model_name} ---")
    print(f"Log-transformed metrics:")
    print(f"  - MSE: {mse:.4f}")
    print(f"  - RMSE: {rmse:.4f}")
    print(f"  - MAE: {mae:.4f}")
    print(f"  - R^2: {r2:.4f}")
    
    print(f"\nOriginal scale metrics:")
    print(f"  - MSE: {mse_orig:.4f}")
    print(f"  - RMSE: {rmse_orig:.4f}")
    print(f"  - MAE: {mae_orig:.4f}")
    print(f"  - R^2: {r2_orig:.4f}")
    
    print(f"\nSoftware engineering specific metrics:")
    print(f"  - MMRE: {mre:.4f} (Mean Magnitude of Relative Error)")
    print(f"  - Pred(0.25): {pred_25:.4f} (Percentage of predictions within 25% of actual)")
    
    # Trả về các chỉ số hiệu suất
    return {
        'model_name': model_name,
        'mse': mse,
        'rmse': rmse,
        'mae': mae,
        'r2': r2,
        'mse_orig': mse_orig,
        'rmse_orig': rmse_orig,
        'mae_orig': mae_orig,
        'r2_orig': r2_orig,
        'mmre': mre,
        'pred_25': pred_25
    }

# Hàm huấn luyện và đánh giá các mô hình
def train_and_evaluate_models(X_train, X_test, y_train, y_test, preprocessor, log_transform=True):
    """
    Huấn luyện và đánh giá các mô hình
    
    Args:
        X_train, X_test, y_train, y_test: Dữ liệu huấn luyện và kiểm thử
        preprocessor: Bộ tiền xử lý dữ liệu
        log_transform: Biến đổi logarithmic đã được áp dụng hay chưa
        
    Returns:
        Dictionary chứa các mô hình và kết quả đánh giá
    """
    # Khởi tạo danh sách mô hình
    models = {
        'Linear Regression': LinearRegression(),
        'Decision Tree': DecisionTreeRegressor(random_state=42),
        'Random Forest': RandomForestRegressor(random_state=42, n_estimators=100)
    }
    
    # Tạo pipeline để kết hợp tiền xử lý và mô hình
    pipelines = {}
    trained_models = {}
    results = []
    
    for name, model in models.items():
        print(f"\nĐang huấn luyện mô hình {name}...")
        pipeline = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('model', model)
        ])
        
        # Huấn luyện mô hình
        pipeline.fit(X_train, y_train)
        
        # Đánh giá mô hình
        result = evaluate_model(pipeline, X_test, y_test, name, log_transform)
        results.append(result)
        
        # Lưu pipeline đã huấn luyện
        pipelines[name] = pipeline
        trained_models[name] = model
    
    return {
        'pipelines': pipelines,
        'models': trained_models,
        'results': results
    }

# Huấn luyện và đánh giá các mô hình
if X is not None and y is not None:
    model_results = train_and_evaluate_models(X_train, X_test, y_train, y_test, preprocessor, log_transform=True)
    
    # So sánh các mô hình
    results_df = pd.DataFrame(model_results['results'])
    print("\nSo sánh hiệu suất các mô hình:")
    print(results_df[['model_name', 'r2_orig', 'mmre', 'pred_25']])

In [ ]:
# Trực quan hóa kết quả dự đoán
def visualize_predictions(models, X_test, y_test, log_transform=True):
    """
    Trực quan hóa kết quả dự đoán của các mô hình
    
    Args:
        models: Dictionary chứa các pipeline đã huấn luyện
        X_test: Dữ liệu kiểm thử
        y_test: Nhãn kiểm thử
        log_transform: Biến đổi logarithmic đã được áp dụng hay chưa
    """
    # Chuyển đổi ngược lại nếu đã áp dụng biến đổi logarithmic
    if log_transform:
        y_test_orig = np.expm1(y_test)
    else:
        y_test_orig = y_test
    
    # Tính toán dự đoán cho từng mô hình
    predictions = {}
    for name, pipeline in models.items():
        y_pred = pipeline.predict(X_test)
        if log_transform:
            predictions[name] = np.expm1(y_pred)
        else:
            predictions[name] = y_pred
    
    # Trực quan hóa giá trị thực tế vs dự đoán
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    for i, (name, y_pred) in enumerate(predictions.items()):
        ax = axes[i]
        ax.scatter(y_test_orig, y_pred, alpha=0.5)
        
        # Vẽ đường y=x (dự đoán hoàn hảo)
        max_value = max(np.max(y_test_orig), np.max(y_pred))
        min_value = min(np.min(y_test_orig), np.min(y_pred))
        ax.plot([min_value, max_value], [min_value, max_value], 'r--')
        
        ax.set_xlabel('Giá trị thực tế')
        ax.set_ylabel('Giá trị dự đoán')
        ax.set_title(f'Actual vs Predicted - {name}')
        
        # Thêm chỉ số R^2
        r2 = r2_score(y_test_orig, y_pred)
        ax.annotate(f'R² = {r2:.4f}', xy=(0.05, 0.95), xycoords='axes fraction',
                   bbox=dict(boxstyle="round,pad=0.3", fc="white", alpha=0.8))
    
    plt.tight_layout()
    plt.show()
    
    # Trực quan hóa phân phối lỗi (residuals)
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    for i, (name, y_pred) in enumerate(predictions.items()):
        ax = axes[i]
        residuals = y_pred - y_test_orig
        ax.scatter(y_pred, residuals, alpha=0.5)
        ax.axhline(y=0, color='r', linestyle='--')
        ax.set_xlabel('Giá trị dự đoán')
        ax.set_ylabel('Residual')
        ax.set_title(f'Residual Plot - {name}')
        
        # Thêm MMRE
        mmre = np.mean(np.abs(residuals) / y_test_orig)
        ax.annotate(f'MMRE = {mmre:.4f}', xy=(0.05, 0.95), xycoords='axes fraction',
                   bbox=dict(boxstyle="round,pad=0.3", fc="white", alpha=0.8))
    
    plt.tight_layout()
    plt.show()

# Trực quan hóa kết quả dự đoán
if X is not None and y is not None and 'pipelines' in model_results:
    visualize_predictions(model_results['pipelines'], X_test, y_test, log_transform=True)

## 5. Tinh chỉnh siêu tham số mô hình (Hyperparameter Tuning)

Để cải thiện hiệu suất của mô hình, chúng ta sẽ thực hiện tinh chỉnh siêu tham số cho mô hình Decision Tree và Random Forest.

In [ ]:
# Tinh chỉnh siêu tham số cho mô hình Decision Tree
def tune_decision_tree(X_train, y_train, X_test, y_test, preprocessor, log_transform=True):
    """
    Tinh chỉnh siêu tham số cho mô hình Decision Tree Regressor
    
    Args:
        X_train, X_test, y_train, y_test: Dữ liệu huấn luyện và kiểm thử
        preprocessor: Bộ tiền xử lý dữ liệu
        log_transform: Biến đổi logarithmic đã được áp dụng hay chưa
        
    Returns:
        Pipeline đã được tinh chỉnh
    """
    # Khởi tạo pipeline
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', DecisionTreeRegressor(random_state=42))
    ])
    
    # Xác định không gian tìm kiếm siêu tham số
    param_grid = {
        'model__max_depth': [None, 5, 10, 15, 20],
        'model__min_samples_split': [2, 5, 10],
        'model__min_samples_leaf': [1, 2, 4, 8]
    }
    
    # Thực hiện tìm kiếm lưới (Grid Search)
    grid_search = GridSearchCV(
        pipeline, param_grid, cv=5,
        scoring='neg_mean_squared_error',
        n_jobs=-1, verbose=1
    )
    
    print("Đang tinh chỉnh siêu tham số cho Decision Tree...")
    grid_search.fit(X_train, y_train)
    
    # In kết quả tốt nhất
    print(f"Siêu tham số tốt nhất: {grid_search.best_params_}")
    print(f"MSE tốt nhất: {-grid_search.best_score_:.4f}")
    
    # Đánh giá mô hình tốt nhất
    best_pipeline = grid_search.best_estimator_
    result = evaluate_model(best_pipeline, X_test, y_test, "Decision Tree (Tuned)", log_transform)
    
    return best_pipeline, result

# Tinh chỉnh siêu tham số cho mô hình Random Forest
def tune_random_forest(X_train, y_train, X_test, y_test, preprocessor, log_transform=True):
    """
    Tinh chỉnh siêu tham số cho mô hình Random Forest Regressor
    
    Args:
        X_train, X_test, y_train, y_test: Dữ liệu huấn luyện và kiểm thử
        preprocessor: Bộ tiền xử lý dữ liệu
        log_transform: Biến đổi logarithmic đã được áp dụng hay chưa
        
    Returns:
        Pipeline đã được tinh chỉnh
    """
    # Khởi tạo pipeline
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', RandomForestRegressor(random_state=42))
    ])
    
    # Xác định không gian tìm kiếm siêu tham số
    param_grid = {
        'model__n_estimators': [50, 100, 200],
        'model__max_depth': [None, 10, 20],
        'model__min_samples_split': [2, 5, 10],
        'model__min_samples_leaf': [1, 2, 4]
    }
    
    # Thực hiện tìm kiếm lưới (Grid Search)
    grid_search = GridSearchCV(
        pipeline, param_grid, cv=5,
        scoring='neg_mean_squared_error',
        n_jobs=-1, verbose=1
    )
    
    print("Đang tinh chỉnh siêu tham số cho Random Forest...")
    grid_search.fit(X_train, y_train)
    
    # In kết quả tốt nhất
    print(f"Siêu tham số tốt nhất: {grid_search.best_params_}")
    print(f"MSE tốt nhất: {-grid_search.best_score_:.4f}")
    
    # Đánh giá mô hình tốt nhất
    best_pipeline = grid_search.best_estimator_
    result = evaluate_model(best_pipeline, X_test, y_test, "Random Forest (Tuned)", log_transform)
    
    return best_pipeline, result

# Thực hiện tinh chỉnh siêu tham số
if X is not None and y is not None:
    # Decision Tree
    dt_tuned_pipeline, dt_tuned_result = tune_decision_tree(X_train, y_train, X_test, y_test, preprocessor, log_transform=True)
    
    # Random Forest
    rf_tuned_pipeline, rf_tuned_result = tune_random_forest(X_train, y_train, X_test, y_test, preprocessor, log_transform=True)
    
    # Thêm kết quả của các mô hình đã tinh chỉnh
    model_results['results'].append(dt_tuned_result)
    model_results['results'].append(rf_tuned_result)
    model_results['pipelines']['Decision Tree (Tuned)'] = dt_tuned_pipeline
    model_results['pipelines']['Random Forest (Tuned)'] = rf_tuned_pipeline
    
    # So sánh tất cả các mô hình
    results_df = pd.DataFrame(model_results['results'])
    print("\nSo sánh hiệu suất tất cả các mô hình:")
    print(results_df[['model_name', 'r2_orig', 'mmre', 'pred_25']].sort_values(by='r2_orig', ascending=False))
    
    # Trực quan hóa kết quả dự đoán của các mô hình đã tinh chỉnh
    tuned_models = {
        'Decision Tree (Tuned)': dt_tuned_pipeline,
        'Random Forest (Tuned)': rf_tuned_pipeline,
        'Linear Regression': model_results['pipelines']['Linear Regression']
    }
    visualize_predictions(tuned_models, X_test, y_test, log_transform=True)

## 6. Thiết kế mô hình COCOMO II mở rộng

Tạo một mô hình ước lượng COCOMO II mở rộng bằng cách kết hợp các biến đầu vào khác nhau.

In [ ]:
# Xây dựng mô hình dự đoán đầy đủ dựa trên COCOMO II
class CocomoIIPredictor:
    """
    Mô hình COCOMO II mở rộng kết hợp dự đoán dựa trên LOC, FP, và UCP
    """
    def __init__(self, model_path=None):
        self.models = {}
        self.preprocessor = None
        self.log_transform = True
        
        if model_path:
            self.load(model_path)
    
    def fit(self, models, preprocessor, log_transform=True):
        """
        Lưu các mô hình đã huấn luyện
        
        Args:
            models: Dictionary chứa các pipeline đã huấn luyện
            preprocessor: Bộ tiền xử lý dữ liệu
            log_transform: Áp dụng biến đổi logarithmic hay không
        """
        self.models = models
        self.preprocessor = preprocessor
        self.log_transform = log_transform
        return self
    
    def predict_effort(self, input_data, model_name='Random Forest (Tuned)'):
        """
        Dự đoán effort dựa trên đầu vào
        
        Args:
            input_data: DataFrame chứa dữ liệu đầu vào
            model_name: Tên mô hình để sử dụng
            
        Returns:
            Giá trị effort dự đoán (người-tháng)
        """
        if model_name not in self.models:
            raise ValueError(f"Mô hình '{model_name}' không tồn tại!")
        
        model = self.models[model_name]
        
        # Dự đoán
        effort_pred = model.predict(input_data)
        
        # Chuyển đổi ngược nếu đã áp dụng biến đổi logarithmic
        if self.log_transform:
            effort_pred = np.expm1(effort_pred)
        
        return effort_pred
    
    def predict_schedule(self, effort):
        """
        Dự đoán lịch trình (thời gian) dựa trên effort
        Sử dụng công thức COCOMO II: TDEV = 3.67 × (PM)^0.28
        
        Args:
            effort: Effort dự đoán (người-tháng)
            
        Returns:
            Thời gian dự kiến (tháng)
        """
        return 3.67 * (effort ** 0.28)
    
    def predict_team_size(self, effort, time):
        """
        Dự đoán kích thước đội ngũ dựa trên effort và thời gian
        
        Args:
            effort: Effort dự đoán (người-tháng)
            time: Thời gian dự kiến (tháng)
            
        Returns:
            Số lượng nhà phát triển
        """
        return np.ceil(effort / time)
    
    def predict_all(self, input_data, model_name='Random Forest (Tuned)'):
        """
        Dự đoán effort, thời gian và kích thước đội ngũ
        
        Args:
            input_data: DataFrame chứa dữ liệu đầu vào
            model_name: Tên mô hình để sử dụng
            
        Returns:
            Dictionary chứa các kết quả dự đoán
        """
        # Dự đoán effort
        effort = self.predict_effort(input_data, model_name)
        
        # Dự đoán lịch trình
        schedule = self.predict_schedule(effort)
        
        # Dự đoán kích thước đội ngũ
        team_size = self.predict_team_size(effort, schedule)
        
        return {
            'effort_pm': effort,
            'time_months': schedule,
            'developers': team_size
        }
    
    def save(self, model_path):
        """
        Lưu mô hình vào file
        
        Args:
            model_path: Đường dẫn thư mục để lưu mô hình
        """
        if not os.path.exists(model_path):
            os.makedirs(model_path)
        
        # Lưu các pipeline
        for name, pipeline in self.models.items():
            file_path = os.path.join(model_path, f"{name.replace(' ', '_')}.pkl")
            joblib.dump(pipeline, file_path)
            print(f"Đã lưu mô hình {name} vào {file_path}")
        
        # Lưu preprocessor
        preprocessor_path = os.path.join(model_path, "preprocessor.pkl")
        joblib.dump(self.preprocessor, preprocessor_path)
        print(f"Đã lưu bộ tiền xử lý vào {preprocessor_path}")
        
        # Lưu cấu hình
        config = {
            'log_transform': self.log_transform,
            'models': list(self.models.keys()),
            'timestamp': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
        }
        config_path = os.path.join(model_path, "config.json")
        with open(config_path, 'w') as f:
            json.dump(config, f, indent=4)
        print(f"Đã lưu cấu hình vào {config_path}")
    
    def load(self, model_path):
        """
        Tải mô hình từ file
        
        Args:
            model_path: Đường dẫn thư mục chứa mô hình đã lưu
        """
        # Tải cấu hình
        config_path = os.path.join(model_path, "config.json")
        if os.path.exists(config_path):
            with open(config_path, 'r') as f:
                config = json.load(f)
            self.log_transform = config['log_transform']
            model_names = config['models']
            
            # Tải preprocessor
            preprocessor_path = os.path.join(model_path, "preprocessor.pkl")
            if os.path.exists(preprocessor_path):
                self.preprocessor = joblib.load(preprocessor_path)
                print(f"Đã tải bộ tiền xử lý từ {preprocessor_path}")
            
            # Tải các mô hình
            for name in model_names:
                file_path = os.path.join(model_path, f"{name.replace(' ', '_')}.pkl")
                if os.path.exists(file_path):
                    self.models[name] = joblib.load(file_path)
                    print(f"Đã tải mô hình {name} từ {file_path}")
                else:
                    print(f"Không tìm thấy mô hình {name} tại {file_path}")
            
            print(f"Đã tải {len(self.models)} mô hình")
        else:
            print(f"Không tìm thấy file cấu hình tại {config_path}")

# Tạo và lưu mô hình COCOMO II mở rộng
if 'pipelines' in model_results:
    cocomo_predictor = CocomoIIPredictor()
    cocomo_predictor.fit(model_results['pipelines'], preprocessor, log_transform=True)
    
    # Lưu mô hình
    cocomo_predictor.save(os.path.join(OUTPUT_DIR, 'cocomo_ii_extended'))
    
    # Ví dụ sử dụng mô hình:
    # Tạo một mẫu dữ liệu đầu vào
    sample_data = X_test.iloc[:5].copy()
    
    print("\nVí dụ sử dụng mô hình COCOMO II mở rộng:")
    print("Dữ liệu đầu vào:")
    print(sample_data)
    
    results = cocomo_predictor.predict_all(sample_data, model_name='Random Forest (Tuned)')
    
    print("\nKết quả dự đoán:")
    for i in range(len(sample_data)):
        print(f"\nMẫu {i+1}:")
        print(f"  - Effort: {results['effort_pm'][i]:.2f} người-tháng")
        print(f"  - Thời gian: {results['time_months'][i]:.2f} tháng")
        print(f"  - Số nhà phát triển: {int(results['developers'][i])}")

## 7. Tạo hàm tiện ích cho người dùng

Tạo các hàm tiện ích để người dùng có thể sử dụng mô hình.

In [ ]:
# Hàm tiện ích cho người dùng
def cocomo_ii_estimate(size, size_type='kloc', model_path=None, model_name='Random Forest (Tuned)'):
    """
    Ước lượng effort, thời gian và kích thước đội ngũ dựa trên kích thước
    
    Args:
        size: Kích thước dự án (KLOC, FP hoặc UCP)
        size_type: Loại kích thước ('kloc', 'fp', 'ucp')
        model_path: Đường dẫn đến mô hình đã lưu
        model_name: Tên mô hình để sử dụng
        
    Returns:
        Dictionary chứa các kết quả dự đoán
    """
    if model_path is None:
        model_path = os.path.join(OUTPUT_DIR, 'cocomo_ii_extended')
    
    # Tải mô hình
    cocomo_predictor = CocomoIIPredictor(model_path)
    
    # Tạo dữ liệu đầu vào
    input_data = pd.DataFrame({
        'schema': [size_type.upper()],
        'size': [size]
    })
    
    # Thêm các cột khác tùy theo loại kích thước
    if size_type == 'kloc':
        input_data['kloc'] = size
        input_data['fp'] = np.nan
        input_data['ucp'] = np.nan
    elif size_type == 'fp':
        input_data['kloc'] = np.nan
        input_data['fp'] = size
        input_data['ucp'] = np.nan
    elif size_type == 'ucp':
        input_data['kloc'] = np.nan
        input_data['fp'] = np.nan
        input_data['ucp'] = size
    else:
        raise ValueError("size_type phải là 'kloc', 'fp', hoặc 'ucp'")
    
    # Dự đoán
    return cocomo_predictor.predict_all(input_data, model_name)

# Tạo giao diện đơn giản để hiển thị kết quả
def display_cocomo_ii_results(size, size_type='kloc', model_name='Random Forest (Tuned)'):
    """
    Hiển thị kết quả ước lượng COCOMO II
    
    Args:
        size: Kích thước dự án (KLOC, FP hoặc UCP)
        size_type: Loại kích thước ('kloc', 'fp', 'ucp')
        model_name: Tên mô hình để sử dụng
    """
    try:
        # Thực hiện ước lượng
        results = cocomo_ii_estimate(size, size_type, model_name=model_name)
        
        # Hiển thị kết quả
        print(f"\n--- COCOMO II Estimation Results ---")
        print(f"Input: {size} {size_type.upper()}")
        print(f"Model: {model_name}")
        print("\nResults:")
        print(f"  - Effort: {results['effort_pm'][0]:.2f} person-months")
        print(f"  - Duration: {results['time_months'][0]:.2f} months")
        print(f"  - Team Size: {int(results['developers'][0])} developers")
        
        # Hiển thị thông báo về chi phí (nếu cần)
        rate_per_month = 5000  # Giả định chi phí trung bình mỗi người/tháng
        cost = results['effort_pm'][0] * rate_per_month
        print(f"\nEstimated Cost (at ${rate_per_month}/person-month): ${cost:.2f}")
        
        return results
    except Exception as e:
        print(f"Error: {str(e)}")
        return None

# Thử nghiệm với một số ví dụ
print("\nThử nghiệm mô hình COCOMO II mở rộng với các ví dụ:")

# Ví dụ với KLOC
print("\nVí dụ với KLOC (Kilo Lines of Code):")
display_cocomo_ii_results(10, 'kloc')

# Ví dụ với FP
print("\nVí dụ với FP (Function Points):")
display_cocomo_ii_results(500, 'fp')

# Ví dụ với UCP
print("\nVí dụ với UCP (Use Case Points):")
display_cocomo_ii_results(300, 'ucp')

## 8. Kết luận

Trong notebook này, chúng ta đã thực hiện xây dựng một mô hình ước lượng nỗ lực phát triển phần mềm dựa trên COCOMO II sử dụng kết hợp các phương pháp học máy. Chúng ta đã:

1. Kết hợp dữ liệu từ 3 schema khác nhau (LOC, FP, UCP)
2. Tiền xử lý dữ liệu cho huấn luyện mô hình
3. Huấn luyện và đánh giá 3 mô hình:
   - Linear Regression (baseline)
   - Decision Tree Regressor
   - Random Forest Regressor
4. Tinh chỉnh các mô hình để cải thiện hiệu suất
5. Xây dựng một bộ dự đoán COCOMO II mở rộng
6. Xuất mô hình dưới dạng file .pkl để sử dụng trong tương lai

Kết quả cho thấy, mô hình Random Forest sau khi tinh chỉnh đạt hiệu suất tốt nhất trong việc dự đoán effort. Mô hình này có thể được sử dụng để ước lượng nỗ lực phát triển phần mềm dựa trên các tham số đầu vào khác nhau như KLOC, FP hoặc UCP.